# Deep Learning Neural Network Models for MGNREGA Wage Prediction

This notebook implements various neural network architectures to predict wages based on MGNREGA dataset features.

## Models Covered:
1. **Simple Feedforward Neural Network (Dense layers)**
2. **Deep Neural Network (Multiple hidden layers)**
3. **LSTM (Long Short-Term Memory) for sequential patterns**
4. **Comparison and Evaluation**

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Deep Learning imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

ImportError: Traceback (most recent call last):
  File "c:\Users\jadit\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

## Step 1: Load and Prepare the Data

In [ ]:
# Load the dataset
# IMPORTANT: Replace the path below with your actual CSV file path
df = pd.read_csv('..\\data\\combined.csv')  # Update this path!

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Check for missing values
print(f"\nMissing values:\n{df.isnull().sum()}")

In [ ]:
# Feature Selection - Using features from your regression analysis
# Based on your previous analysis, these were the important features

feature_columns = [
    'Total_Exp',
    'Approved_Labour_Budget',
    'Women_Persondays',
    'Persondays_of_Central_Liability_so_far',
    'Total_Households_Worked',
    'Total_Individuals_Worked',
    'Number_of_Completed_Works',
    'Number_of_Ongoing_Works'
]

target_column = 'Wages'

# Create feature matrix X and target vector y
X = df[feature_columns].copy()
y = df[target_column].copy()

# Handle missing values
X = X.fillna(X.median())
y = y.fillna(y.median())

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

## Step 2: Train-Test Split and Standardization

In [ ]:
# Split data into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Standardize features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to numpy arrays
X_train_scaled = np.array(X_train_scaled, dtype=np.float32)
X_test_scaled = np.array(X_test_scaled, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

print(f"\nFeature statistics after scaling:")
print(f"Mean: {X_train_scaled.mean():.4f}")
print(f"Std: {X_train_scaled.std():.4f}")

## Model 1: Simple Feedforward Neural Network (Baseline)

This is a basic dense neural network with:
- Input layer
- Two hidden layers with ReLU activation
- Output layer for regression

In [ ]:
# Build Simple Neural Network
def build_simple_nn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim, name='hidden_1'),
        Dropout(0.2),
        Dense(32, activation='relu', name='hidden_2'),
        Dropout(0.2),
        Dense(1, name='output')  # Single output for regression
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae', 'mse']
    )
    
    return model

# Create the model
simple_nn = build_simple_nn(X_train_scaled.shape[1])

# Display model architecture
print("Simple Neural Network Architecture:")
simple_nn.summary()

In [ ]:
# Set up callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=0.00001,
    verbose=1
)

# Train the model
print("\n" + "="*50)
print("Training Simple Neural Network...")
print("="*50 + "\n")

history_simple = simple_nn.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
# Evaluate Simple NN
y_pred_simple = simple_nn.predict(X_test_scaled).flatten()

mse_simple = mean_squared_error(y_test, y_pred_simple)
rmse_simple = np.sqrt(mse_simple)
mae_simple = mean_absolute_error(y_test, y_pred_simple)
r2_simple = r2_score(y_test, y_pred_simple)

print("\n" + "="*50)
print("Simple Neural Network - Test Results")
print("="*50)
print(f"MSE: {mse_simple:,.2f}")
print(f"RMSE: {rmse_simple:,.2f}")
print(f"MAE: {mae_simple:,.2f}")
print(f"R² Score: {r2_simple:.4f}")
print("="*50)

## Model 2: Deep Neural Network (More Layers)

A deeper architecture with:
- More hidden layers
- Batch normalization
- Dropout for regularization

In [ ]:
from tensorflow.keras.layers import BatchNormalization

# Build Deep Neural Network
def build_deep_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim, name='hidden_1'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(64, activation='relu', name='hidden_2'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(32, activation='relu', name='hidden_3'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(16, activation='relu', name='hidden_4'),
        Dropout(0.2),
        
        Dense(1, name='output')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae', 'mse']
    )
    
    return model

# Create the model
deep_nn = build_deep_nn(X_train_scaled.shape[1])

print("Deep Neural Network Architecture:")
deep_nn.summary()

In [ ]:
# Train Deep NN
print("\n" + "="*50)
print("Training Deep Neural Network...")
print("="*50 + "\n")

history_deep = deep_nn.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
# Evaluate Deep NN
y_pred_deep = deep_nn.predict(X_test_scaled).flatten()

mse_deep = mean_squared_error(y_test, y_pred_deep)
rmse_deep = np.sqrt(mse_deep)
mae_deep = mean_absolute_error(y_test, y_pred_deep)
r2_deep = r2_score(y_test, y_pred_deep)

print("\n" + "="*50)
print("Deep Neural Network - Test Results")
print("="*50)
print(f"MSE: {mse_deep:,.2f}")
print(f"RMSE: {rmse_deep:,.2f}")
print(f"MAE: {mae_deep:,.2f}")
print(f"R² Score: {r2_deep:.4f}")
print("="*50)

## Model 3: LSTM Neural Network

LSTM (Long Short-Term Memory) networks can capture temporal dependencies.
Here we reshape the data to use LSTM architecture.

In [ ]:
# Reshape data for LSTM (samples, timesteps, features)
# We'll treat each feature as a timestep
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

print(f"LSTM input shape: {X_train_lstm.shape}")

# Build LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape, name='lstm_1'),
        Dropout(0.3),
        
        LSTM(32, return_sequences=False, name='lstm_2'),
        Dropout(0.3),
        
        Dense(16, activation='relu', name='hidden_1'),
        Dropout(0.2),
        
        Dense(1, name='output')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=['mae', 'mse']
    )
    
    return model

# Create LSTM model
lstm_model = build_lstm_model((X_train_lstm.shape[1], X_train_lstm.shape[2]))

print("\nLSTM Neural Network Architecture:")
lstm_model.summary()

In [ ]:
# Train LSTM
print("\n" + "="*50)
print("Training LSTM Neural Network...")
print("="*50 + "\n")

history_lstm = lstm_model.fit(
    X_train_lstm, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
# Evaluate LSTM
y_pred_lstm = lstm_model.predict(X_test_lstm).flatten()

mse_lstm = mean_squared_error(y_test, y_pred_lstm)
rmse_lstm = np.sqrt(mse_lstm)
mae_lstm = mean_absolute_error(y_test, y_pred_lstm)
r2_lstm = r2_score(y_test, y_pred_lstm)

print("\n" + "="*50)
print("LSTM Neural Network - Test Results")
print("="*50)
print(f"MSE: {mse_lstm:,.2f}")
print(f"RMSE: {rmse_lstm:,.2f}")
print(f"MAE: {mae_lstm:,.2f}")
print(f"R² Score: {r2_lstm:.4f}")
print("="*50)

## Model Comparison and Visualization

In [ ]:
# Create comparison dataframe
results_df = pd.DataFrame({
    'Model': ['Simple NN', 'Deep NN', 'LSTM'],
    'MSE': [mse_simple, mse_deep, mse_lstm],
    'RMSE': [rmse_simple, rmse_deep, rmse_lstm],
    'MAE': [mae_simple, mae_deep, mae_lstm],
    'R² Score': [r2_simple, r2_deep, r2_lstm]
})

print("\n" + "="*70)
print("MODEL COMPARISON - ALL NEURAL NETWORKS")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)

# Find best model
best_model_idx = results_df['R² Score'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']
best_r2 = results_df.loc[best_model_idx, 'R² Score']

print(f"\n🏆 BEST MODEL: {best_model_name} with R² = {best_r2:.4f}")

In [ ]:
# Visualization 1: Model Comparison Bar Chart
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# MSE Comparison
axes[0, 0].bar(results_df['Model'], results_df['MSE'], color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0, 0].set_title('Mean Squared Error (Lower is Better)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('MSE')
axes[0, 0].grid(axis='y', alpha=0.3)

# RMSE Comparison
axes[0, 1].bar(results_df['Model'], results_df['RMSE'], color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0, 1].set_title('Root Mean Squared Error (Lower is Better)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].grid(axis='y', alpha=0.3)

# MAE Comparison
axes[1, 0].bar(results_df['Model'], results_df['MAE'], color=['skyblue', 'lightgreen', 'lightcoral'])
axes[1, 0].set_title('Mean Absolute Error (Lower is Better)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('MAE')
axes[1, 0].grid(axis='y', alpha=0.3)

# R² Score Comparison
axes[1, 1].bar(results_df['Model'], results_df['R² Score'], color=['skyblue', 'lightgreen', 'lightcoral'])
axes[1, 1].set_title('R² Score (Higher is Better)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('R² Score')
axes[1, 1].set_ylim([0, 1])
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 2: Training History (Loss over epochs)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Simple NN
axes[0].plot(history_simple.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history_simple.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_title('Simple NN - Training History', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Deep NN
axes[1].plot(history_deep.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history_deep.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_title('Deep NN - Training History', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss (MSE)')
axes[1].legend()
axes[1].grid(alpha=0.3)

# LSTM
axes[2].plot(history_lstm.history['loss'], label='Training Loss', linewidth=2)
axes[2].plot(history_lstm.history['val_loss'], label='Validation Loss', linewidth=2)
axes[2].set_title('LSTM - Training History', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Loss (MSE)')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 3: Actual vs Predicted for all models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Simple NN
axes[0].scatter(y_test, y_pred_simple, alpha=0.5, s=20)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Wages')
axes[0].set_ylabel('Predicted Wages')
axes[0].set_title(f'Simple NN (R²={r2_simple:.4f})', fontweight='bold')
axes[0].grid(alpha=0.3)

# Deep NN
axes[1].scatter(y_test, y_pred_deep, alpha=0.5, s=20, color='green')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Wages')
axes[1].set_ylabel('Predicted Wages')
axes[1].set_title(f'Deep NN (R²={r2_deep:.4f})', fontweight='bold')
axes[1].grid(alpha=0.3)

# LSTM
axes[2].scatter(y_test, y_pred_lstm, alpha=0.5, s=20, color='coral')
axes[2].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[2].set_xlabel('Actual Wages')
axes[2].set_ylabel('Predicted Wages')
axes[2].set_title(f'LSTM (R²={r2_lstm:.4f})', fontweight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Key Findings and Recommendations

### Summary:
1. **Model Performance**: All three neural network architectures were trained and evaluated
2. **Best Model**: The model comparison shows which architecture works best for this dataset
3. **Deep Learning Advantages**:
   - Can capture non-linear relationships
   - Automatically learns feature interactions
   - Scalable to larger datasets

### When to Use Each Model:
- **Simple NN**: Good baseline, fast training, interpretable
- **Deep NN**: Better for complex patterns, more data needed
- **LSTM**: Best for sequential/temporal patterns

### Next Steps:
1. **Hyperparameter Tuning**: Adjust learning rate, batch size, neurons
2. **Feature Engineering**: Create new features from existing ones
3. **Ensemble Methods**: Combine predictions from multiple models
4. **Cross-Validation**: Use k-fold CV for more robust evaluation

In [ ]:
# Save the best model
# Uncomment the line below to save
# best_model.save('best_mgnrega_wage_prediction_model.h5')
print("\nTo save the model, uncomment the save line in this cell")